In [61]:
import sys
sys.path.append('../')
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlotting

In [62]:
from infrastucture.instrument_collection import InstrumentCollection as ic

In [63]:
pair ="GBP_USD"
granularity = "H4"

df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")

MAL_LIST = [10, 20, 50, 100, 200]

In [64]:
df.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

COPIO LE COLONNE CHE MI SERVONO PER CACLOLARE LA MEDIA 

In [65]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', ]].copy()

In [66]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c
0,2019-10-25 13:00:00+00:00,1.28127,1.28438,1.28044,1.28185
1,2019-10-25 17:00:00+00:00,1.28182,1.28367,1.28152,1.28288
2,2019-10-27 21:00:00+00:00,1.28150,1.28342,1.28150,1.28187
3,2019-10-28 01:00:00+00:00,1.28184,1.28209,1.28116,1.28160
4,2019-10-28 05:00:00+00:00,1.28156,1.28594,1.28148,1.28262
5,2019-10-28 09:00:00+00:00,1.28266,1.28563,1.28225,1.28334
6,2019-10-28 13:00:00+00:00,1.28338,1.28766,1.28294,1.28568
7,2019-10-28 17:00:00+00:00,1.28570,1.28709,1.28544,1.28593
8,2019-10-28 21:00:00+00:00,1.28580,1.28620,1.28448,1.28508
9,2019-10-29 01:00:00+00:00,1.28504,1.28536,1.28351,1.28508


Calcolo la media dei valori di una colonna

In [67]:
for ma in MAL_LIST:
    df_ma[f"MA_{ma}"] = df_ma.mid_c.rolling(window=ma).mean()


# elimina i valori nulli
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)


In [68]:
df_plot = df_ma.iloc[:500]

In [69]:
cp = CandlePlotting(df_plot)

In [70]:
medie = [ f'MA_{ma}' for ma in MAL_LIST ]



In [71]:

cp.show_plot(line_traces=medie)

In [72]:
MA_S = "MA_10"
MA_L = "MA_20"
BUY = 1
SELL = -1
NONE = 0

In [73]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L ]].copy()


In [75]:
# calcolo la differenza tra le medie
df_an['DELTA'] = df_an.MA_10 - df_an.MA_20

# ottengo il delta precedente 
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)


In [76]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    else:
        return NONE

In [77]:
# aggiungo n campo trade 
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [78]:
df_trade = df_an[df_an.TRADE != NONE].copy()

In [79]:
df_trade.shape

(202, 10)

In [80]:
cp = CandlePlotting(df_an.iloc[15:65])

cp.show_plot(line_traces=[MA_L, MA_S])

In [81]:
ic.LoadInstrument(ic, "../Data" )

In [82]:
ic.instrument_dict[pair]

{'name': 'GBP_USD', 'ins_type': 'CURRENCY', 'displayName': 'GBP/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.0333}

In [83]:
ins_data = ic.instrument_dict[pair]

In [84]:
df_trade['DIFF'] = df_trade.mid_c.diff().shift(-1)
df_trade.fillna(0, inplace=True)

In [85]:
df_trade['PIPS']  = df_trade['DIFF'] / ins_data.pipLocation 

df_trade['PIPS'] = df_trade['PIPS'] * df_trade['TRADE']

In [86]:
df_trade.PIPS.sum()

2713.500000000004

In [87]:
df_trade['GAIN_C'] = df_trade['PIPS'].cumsum()

In [88]:
cp = CandlePlotting(df_trade, candele=False)
cp.show_plot(line_traces=['GAIN_C'])